In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import uuid


driver = webdriver.Chrome()
URL = "https://www.metacritic.com/movie/the-matrix"
driver.get(URL)

delay = 10

print (driver)

time.sleep(1)

try: # if the reject cookies button is there then click it
    reject_cookies_btn = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="onetrust-reject-all-handler"]')))
    reject_cookies_btn.click()

except NoSuchElementException: # open the cookies settings, then click reject
    cookies_settings_btn = driver.find_element(by=By.XPATH, value='//*[@id="ot-sdk-btn"]')
    cookies_settings_btn.click()
    reject_cookies_btn = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="onetrust-pc-sdk"]/div[3]/div[1]/button[1]')))
    reject_cookies_btn.click()

except TimeoutException:
    print ("Loading cookies took too much time!")

except:
    pass

def save_raw_data( data): #TODO use try, except block

    if os.path.isdir('raw_data'):
        if os.path.exists(os.path.join(os.getcwd(), 'raw_data', data['friend_id'])):
            with open(data['friend_id']+ ".json", "w") as file:
                file.write(data)

    else:
        create_data_folder( data)

def create_data_folder( data):

    if not os.path.isdir('raw_data'):
        os.makedirs('raw_data')
        save_raw_data(data)


def get_image_src(driver):
    img_src = driver.find_element(By.XPATH, value = '//img[@class="summary_img"]').get_attribute("src")
    return img_src

# movies = driver.find_elements(By.XPATH, '//div[@class="title_bump"]//a[@class="title"]')
# links = [movie.get_attribute('href') for movie in movies]
src = get_image_src(driver)
print(src)

data = {'uuid':[],'friend_id':[],'title':[],'metascore':[],'release_date':[],'starring':[],
                'director':[],'genres':[],'rating':[],'runtime':[],'images':[]}

data['uuid'] = str(uuid.uuid4())

last_part_url = driver.current_url.split("/")[-1]
data['friend_id'] = last_part_url

data['title'] = driver.find_element(By.XPATH, value = '//div[@class="product_page_title oswald"]/h1').text
print(data['title'])

#data['metascore'] = driver.find_element(By.XPATH, value = '//span[contains(@class, "metascore_w larger"]').text
data['metascore'] = driver.find_element(By.XPATH, value = '//a[@class="metascore_anchor"]/span').text
print(data['metascore'])

data['release_date'] = driver.find_element(By.XPATH, value = '//span[@class="release_date"]/span[2]').text
print(data['release_date'])

actors = driver.find_elements(By.XPATH, value = '//div[@class="summary_cast details_section"]/span[2]/a')
data['starring'] = [actor.text for actor in actors]
#for actor in actors: data['starring'].append(actor.text) 
print (data['starring'])

directors = driver.find_elements(By.XPATH, value = '//div[@class="director"]/a')
# data['director'] = driver.find_element(By.XPATH, value = '//div[@class="director"]/a').text
data['director'] = [director.text for director in directors]
print (data['director'])

genres = driver.find_elements(By.XPATH, value = '//div[@class="genres"]/span[2]/span')
data['genres'] = [genre.text for genre in genres]
print (data['genres'])

rating = driver.find_element(By.XPATH, value = '//div[@class="rating"]/span[2]').text
data['rating'] = rating
print (rating)

runtime = driver.find_element(By.XPATH, value = '//div[@class="runtime"]/span[2]').text
data['runtime']  = runtime
print (runtime)

print (data)












<selenium.webdriver.chrome.webdriver.WebDriver (session="7e589cb6c4c17bb368d66666eefb64a8")>
https://static.metacritic.com/images/products/movies/5/14d38f138eb320954cd1e07d0449e5a6-250h.jpg
The Matrix
73
March 31, 1999
['Carrie-Anne Moss', 'Keanu Reeves', 'Laurence Fishburne']
['Lana Wachowski', 'Lilly Wachowski']
['Action', 'Adventure', 'Sci-Fi', 'Thriller']
R
136 min
{'uuid': '992ae5b5-d115-4c48-984d-b73757f17f2b', 'friend_id': 'the-matrix', 'title': 'The Matrix', 'metascore': '73', 'release_date': 'March 31, 1999', 'starring': ['Carrie-Anne Moss', 'Keanu Reeves', 'Laurence Fishburne'], 'director': ['Lana Wachowski', 'Lilly Wachowski'], 'genres': ['Action', 'Adventure', 'Sci-Fi', 'Thriller'], 'rating': 'R', 'runtime': '136 min', 'images': []}


In [21]:
import pathlib
import os
import json
import requests

nice_data = {'uuid': '992ae5b5-d115-4c48-984d-b73757f17f2b', 'friend_id': 'the-matrix', 'title': 'The Matrix', 'metascore': '73', 'release_date': 'March 31, 1999', 'starring': ['Carrie-Anne Moss', 'Keanu Reeves', 'Laurence Fishburne'], 'director': ['Lana Wachowski', 'Lilly Wachowski'], 'genres': ['Action', 'Adventure', 'Sci-Fi', 'Thriller'], 'rating': 'R', 'runtime': '136 min', 'images': ['https://static.metacritic.com/images/products/movies/5/14d38f138eb320954cd1e07d0449e5a6-250h.jpg']}

def save_raw_data( data): #TODO use try, except block

    if os.path.isdir('raw_data'):
        filename = "raw_data/" + data['friend_id'] + ".json"
        print(filename)
        with open(filename, "w") as file:
            file.write(json.dumps(data))

    else:
        create_data_folder( data)

def create_data_folder( data):

    if not os.path.isdir('raw_data'):
        os.makedirs('raw_data')
        save_raw_data(data)

def save_image(data):

    for x, image in enumerate(data['images']):
        filename = "raw_data/images/{}_{}.jpg".format(data['friend_id'], str(x))
        img_data = requests.get(image).content
        with open(filename, 'wb') as handler:
            handler.write(img_data)



save_image(nice_data)


TypeError: a bytes-like object is required, not 'Response'